## Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import  NoSuchElementException,ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep
import os


## Cấu hình tùy chọn cho Edge

In [2]:
options = Options()
options.add_argument("--start-maximized")  # Mở ở chế độ toàn màn hình
options.add_argument("--inprivate")  # Chế độ ẩn danh (InPrivate mode)

## Đăng nhập

In [3]:
driver = webdriver.Edge(options=options)
driver.get("https://www.facebook.com/groups/ued.confessions?locale=vi_VN")
username = driver.find_element(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .xod5an3 input')
password = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg input')[0]
login = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg')[1]
username.send_keys('khoalolriot@gmail.com')
password.send_keys('Akhoa@6204')
login.click()

## Hàm scroll

In [4]:
def scroll():
    driver.execute_script("window.scrollTo({top: document.body.scrollHeight, behavior: 'smooth'});")
    sleep(1)
def scroll_to_post(post, time):
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", post)
    sleep(time)

## Đường dẫn lưu file

In [5]:
path_normal = 'D:/kpw/crawlFB/post_facebook.csv'
path_reel = 'D:/kpw/crawlFB/post_reel.csv'

## Lưu kết quả

In [6]:
def saveOrCreate(result_post_normal, result_post_reel):
    # Kiểm tra và tạo file CSV cho post_facebook nếu không tồn tại
    if not os.path.exists(path_normal):
        os.makedirs(os.path.dirname(path_normal), exist_ok=True)
        df_normal = pd.DataFrame(columns=['index', 'linkPoster', 'name', 'role', 'time', 'content', 'content_img', 'react_list'])
        df_normal.to_csv(path_normal, index=False)

    # Nếu có dữ liệu mới, thêm vào file post_facebook.csv
    if result_post_normal:
        new_post_normal = pd.DataFrame(result_post_normal)
        df_normal = pd.read_csv(path_normal)
        df_normal = pd.concat([df_normal, new_post_normal], ignore_index=True)
        df_normal.to_csv(path_normal, index=False)

    # Kiểm tra và tạo file CSV cho post_reel nếu không tồn tại
    if not os.path.exists(path_reel):
        os.makedirs(os.path.dirname(path_reel), exist_ok=True)
        df_reel = pd.DataFrame(columns=['index', 'linkPoster', 'name', 'time', 'content', 'linkVideo', 'typeVideo'])
        df_reel.to_csv(path_reel, index=False)

    # Nếu có dữ liệu mới, thêm vào file post_reel.csv
    if result_post_reel:
        new_post_reel = pd.DataFrame(result_post_reel)
        df_reel = pd.read_csv(path_reel)
        df_reel = pd.concat([df_reel, new_post_reel], ignore_index=True)
        df_reel.to_csv(path_reel, index=False)


## Lấy ra post đầu tiên của lần crawl trước

In [7]:
def get_first_post():
    # Đọc dữ liệu từ các file CSV
    df_normal = pd.read_csv(path_normal)
    df_reel = pd.read_csv(path_reel)

    # Kiểm tra nếu cả hai dataframe đều rỗng
    if df_normal.empty and df_reel.empty:
        return None
    
    # Nếu df_normal không rỗng
    if not df_normal.empty:
        first_row_normal = df_normal.iloc[0]
    else:
        first_row_normal = None

    # Nếu df_reel không rỗng
    if not df_reel.empty:
        first_row_reel = df_reel.iloc[0]
    else:
        first_row_reel = None

    # So sánh và trả về dòng có index nhỏ hơn nếu cả hai dataframe không rỗng
    if first_row_normal is not None and first_row_reel is not None:
        if first_row_normal['index'] > first_row_reel['index']:
            result = first_row_reel
        else:
            result = first_row_normal
    elif first_row_normal is not None:
        result = first_row_normal
    elif first_row_reel is not None:
        result = first_row_reel
    else:
        result = None

    return result


In [8]:
result_post_normal = []
result_post_reel = []

## Main

In [ ]:
saveOrCreate(None, None)
index  = 0
post_list = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
prev_result = get_first_post()
result_post_normal=  []
result_post_reel = []
while True:
    post_data_normal = {}
    post_data_reel = {} 
    try:
        close = driver.find_element(By.XPATH,'//*[@id="facebook"]/body/div[6]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[3]')
        close.click()
        sleep(5)
    except NoSuchElementException: 
        pass  
    
    if index >= len(post_list):  
        scroll()
        post_list_new = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
        if len(post_list_new) == len(post_list):
            print()
            print('Thu thập được:',len(post_list),'bài đăng')
            break 
        else:
            post_list = post_list_new 
    
    try:
        scroll_to_post(post_list[index], 1)
        print(f"Đã scroll tới bài đăng số {index + 1}:")
        print('post normal')
        main  = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
        
        info_box  = main.find_element(By.XPATH,'./div[2]')            
        content_box = main.find_element(By.XPATH,'./div[3]')
        react_cmt_box = main.find_element(By.XPATH,'./div[4]')
        
        user_link = info_box.find_element(By.XPATH, './div/div[2]/div/div[2]').find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        
        user_name = info_box.find_element(By.XPATH, './div/div[2]/div/div[1]').find_element(By.CSS_SELECTOR, '.xt0psk2').text  
        
        time_post = info_box.find_element(By.XPATH, './div/div[2]/div/div[2]')

        try: 
            role = time_post.find_element(By.CSS_SELECTOR,'.x3nfvp2.x1kgmq87').text
        except NoSuchElementException:
            role = None

        try: 
            time_link = time_post.find_element(By.CSS_SELECTOR, 'use').get_attribute('xlink:href')
            time_post = driver.find_element(By.CSS_SELECTOR, f'text{time_link}').get_attribute('textContent')
        except NoSuchElementException:  
            time_post = time_post.text.replace('·','').strip()

        try: 
            main_content = content_box.find_element(By.XPATH, './div[1]')
            try: 
                more_content = main_content.find_element(By.CSS_SELECTOR, 'div[role="button"]')
                if more_content.text.strip() == 'Xem thêm': 
                    try:
                        scroll_to_post(more_content, 0)
                        more_content.click()
                    except ElementClickInterceptedException: 
                        driver.execute_script("window.scrollBy({top: -400, behavior: 'smooth'});")
                        sleep(1)
                        more_content.click()
            except NoSuchElementException: 
                pass
            main_content = content_box.find_element(By.XPATH, './div[1]').text
        except NoSuchElementException:
            main_content = None

        content_img_post = {}        
        try:     
            content_img = content_box.find_element(By.XPATH, './div[2]')
            content_img_list = content_img.find_elements(By.CSS_SELECTOR, 'img')
            for i, img in enumerate(content_img_list):
                content_img_post[i] = img.get_attribute('src')
        except NoSuchElementException: 
            pass
        

        react_list = {
            "Thích" : 0,
            'Yêu thích' : 0,
            'Thương thương': 0,
            'Haha': 0,
            'Wow': 0,
            'Buồn': 0,
            'Phẫn nộ': 0
        }
        try: 
            react = react_cmt_box.find_element(By.XPATH,'./div/div/div[1]')

            total_cmt = react.find_element(By.XPATH,'./div/div[1]/div/div[2]').text

            total_react = react.find_element(By.XPATH,'./div/div[1]/div/div[1]/div') 

            react_icon = react.find_elements(By.XPATH,'./div/div[1]/div/div[1]/span/span/span')
            for react in react_icon: 
                rs = react.find_element(By.CSS_SELECTOR,'div[aria-label]').get_attribute('aria-label').split(':')[0].strip()
                for i in react_list: 
                    if i== rs:
                        react_list[i] = react.find_element(By.CSS_SELECTOR,'div[aria-label]').get_attribute('aria-label').split(':')[1].strip().split()[0]
        except NoSuchElementException:
            pass
        post_data_normal = {
            'index': index + 1,
            'linkPoster': user_link if user_name != 'Người tham gia ẩn danh' else 'No link',
            'name': user_name,
            'role': role if role else 'Thành viên',
            'time': time_post,
            'content': main_content if main_content else 'No content',
            'content_img': content_img_post if content_img_post else 'No picture',
            'react_list': react_list
        }
        print(post_data_normal['react_list'])
        index += 1
    except NoSuchElementException: # video
        scroll_to_post(post_list[index], 1)
        print(f"Đã scroll tới bài đăng số {index + 1}:")
        print('post video')
        
        main = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')

        content_box = main.find_element(By.XPATH,'./div[2]')
        
        link = content_box.find_element(By.CSS_SELECTOR, 'a')
        href = link.get_attribute('href')
        
        box = link.find_element(By.XPATH,'./div[1]/div[3]/div/div/div[1]/div/div/div/div/div/div[2]/span/span')
        
        linkPoster = box.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
        
        box = box.text.split('·')
        typeVideo = box[0].replace('\n', '').strip()
        name = box[1].replace('\n', '').strip()
        time = box[2].replace('\n', '').strip()
        
        try: 
            main_content = link.find_element(By.XPATH,'./div[1]/div[2]/div/div/div[2]/span/div')
            try: 
                more_content = main_content.find_element(By.CSS_SELECTOR, 'div[role="button"]')
                if more_content.text.strip() == 'Xem thêm': 
                    scroll_to_post(more_content, 0)
                    more_content.click()
            except NoSuchElementException: 
                pass
            main_content = link.find_element(By.XPATH, './div[1]/div[2]/div/div/div[2]/span/div').text
        except NoSuchElementException:
            main_content = None
        
        post_data_reel = {
            'index': index + 1,
            'linkPoster': linkPoster,
            'name': name,
            'time': time,
            'content': main_content if main_content else 'No content',
            'linkVideo': href,
            'typeVideo': typeVideo,
        }
        
        index += 1
        print(post_data_reel)
    
    if prev_result is None: 
        pass
    else:
        if prev_result == post_data_normal: 
            break
        if prev_result == post_data_reel:
            break
    if post_data_normal:  
        result_post_normal.append(post_data_normal)
    if post_data_reel:
        result_post_reel.append(post_data_reel)
    sleep(1)
print('Crawl thành công')


In [ ]:
saveOrCreate(result_post_normal, result_post_reel)
print('Save thành công')

In [26]:
post_list = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
index =  0

main  = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
scroll_to_post(main, 0)
react_cmt_box = main.find_element(By.XPATH,'./div[4]')
try:
    more_cmt = react_cmt_box.find_element(By.XPATH, './div/div/div[@class="xdj266r xat24cr x1d52u69 xktsk01"]')
    if 'Xem thêm' in more_cmt.text:
        try:
            scroll_to_post(more_cmt, 0)
            more_cmt.click()
        except: 
            driver.execute_script("window.scrollBy({top: -400, behavior: 'smooth'});")
            more_cmt.click()

except NoSuchElementException:
    print('không có')
    pass



/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div[3]/div
/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div[3]/div[8]
/div/div[1]/div/div[2]/div[2]/div[1]/div/div

/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div[3]/div[8]/div/div[1]/div/div[2]/div[2]/div[1]/div/div/div

In [ ]:
post_data_cmt = []
cmt_box = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div[3]')
scroll_to_post(cmt_box, 1)
list_cmt = cmt_box.find_elements(By.XPATH, "./div[position() < last()]")
for index, cmt in enumerate(list_cmt):
    scroll_to_post(cmt, 1)
    
    try:
        content =  cmt.find_element(By.XPATH, './div/div[1]/div/div[2]/div[1]/div[1]/div/div/div').text 
    except NoSuchElementException:
        content = cmt.find_element(By.XPATH, './div/div[1]/div/div[2]/div[2]/div[1]/div/div/div').text
    try:
        name = cmt.find_element(By.XPATH, './div/div[1]/div/div[2]/div[1]/div[1]/div/div/span').text
    except NoSuchElementException:
        name = cmt.find_element(By.XPATH, './div/div[1]/div/div[2]/div[2]/div[1]/div/div/span').text

    sub_cmt = {
        'name': name, 
        'content': content,
        'id': index + 1
    }
    print(sub_cmt)
    post_data_cmt.append(sub_cmt)

    while True:
        try:
            more_cmt_button_list = cmt.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
            has_clicked = False 
            for button in more_cmt_button_list:
                if "Xem" in button.text and "phản hồi" in button.text:
                    button.click()
                    sleep(4)
                    has_clicked = True
                    break  
            if not has_clicked:
                break
        except Exception:
            break
            
    try:
        button = cmt.find_element(By.XPATH, f'./div/div[2]').click()
        xpath = './div/div[2]'
        more = '/div/div/div[1]'
        while True:
            try:
                more_cmt = cmt.find_element(By.XPATH, f'{xpath + more}')
                print(more_cmt.text)
                
                print('end')
            except NoSuchElementException:
                break 
            xpath += '/div/div/div[2]'
            more = '/div/div/div[2]'
        print(more_cmt.text)
    except: pass 



{'name': 'Quốc Vũ', 'content': 'Ib tư vấn vào việc luôn\nỨng tuyển cho bạn mình', 'id': 1}
{'name': 'Yến Nhi', 'content': 'Hoàng Nguyễnvế trên thì ổn còn về sau không ổn tí nào đúng không Tuyết NhiTrương Công HuySương LêNguyễn Lữ Kiệt', 'id': 2}
{'name': 'Phạm Hưng', 'content': 'ứng tuyển', 'id': 3}
{'name': 'Nguyễn Gia Huy', 'content': 'Buồn quá mik mắc chứng ngại giao tiêpd', 'id': 4}
{'name': 'An Lê', 'content': 'Minh Vũ Trương Văn mời 2 bạn', 'id': 5}
{'name': 'Phạm Cường', 'content': 'mình', 'id': 6}
{'name': 'Đặng Thanh Phú', 'content': 'K cần phí mình tới ăn xong về là đc ạ', 'id': 7}
{'name': 'Thơm', 'content': 'Mai Văn Long bạn dki di r về chia cát xê cho toai nha ~', 'id': 8}
{'name': 'Nguyễn Vũ Kiều Giang', 'content': 'ứng tuyển Đặng Hiếu zô vị trí', 'id': 9}
{'name': 'Văn Hồngg', 'content': 'Tiến Vượng đki đi em', 'id': 10}
{'name': 'Rin Trương', 'content': 'Em ưng tuyển ạ\nQuốc Vũ', 'id': 11}
{'name': 'Ngô Quốc Tuấn', 'content': 'Ứng mình ko lấy tiền', 'id': 12}
{'name': '